In [2]:
import duckdb

In [3]:
con = duckdb.connect(database = "csecicids2018.duckdb", read_only = False)

In [6]:
con.execute('CREATE OR REPLACE TABLE flows AS SELECT "Flow ID", CAST(Timestamp AS VARCHAR), Label FROM read_csv_auto("../datasets/CSECICIDS2018_improved/*.csv");')

In [7]:
con.execute("UPDATE Flows SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [8]:
con.execute('SELECT DISTINCT Label FROM flows')
Labels = con.fetch_df()
Labels

,Label
0,DDoS-HOIC
1,Web Attack - XSS
2,Infiltration - NMAP Portscan
3,Web Attack - Brute Force
4,BENIGN
5,DDoS-LOIC-UDP
6,Botnet Ares
7,Infiltration - Communication Victim Attacker
8,DoS Slowloris
9,DoS GoldenEye


In [9]:
con.execute('SELECT Label, COUNT(*) AS count FROM Flows GROUP BY Label')
counted_labels = con.fetch_df()
counted_labels

,Label,count
0,DDoS-HOIC,1082293
1,Web Attack - XSS,113
2,Infiltration - NMAP Portscan,89374
3,Web Attack - Brute Force,131
4,BENIGN,59659723
5,Botnet Ares,142921
6,DDoS-LOIC-UDP,2527
7,Infiltration - Communication Victim Attacker,204
8,DoS Slowloris,8490
9,SSH-BruteForce,94197


In [10]:
con.execute('CREATE OR REPLACE TABLE sampled_flows ("Flow_ID" VARCHAR, Timestamp DATE, Label VARCHAR)')

In [13]:
for label in Labels.to_numpy():
    con.execute('WITH tab1 AS (SELECT "Flow ID", Timestamp, Label FROM flows WHERE Label = \'{}\' ORDER BY RANDOM() LIMIT 2000) INSERT INTO sampled_flows SELECT * FROM tab1'.format(label[0]))

BinderException: Binder Error: Referenced column "Timestamp" not found in FROM clause!
Candidate bindings: "flows.Label"

In [9]:
con.execute('SELECT Label, COUNT(*) AS count FROM sampled_flows GROUP BY Label')
con.fetchall()

[('DDoS-HOIC', 2000),
 ('Web Attack - XSS', 113),
 ('Infiltration - NMAP Portscan', 2000),
 ('Web Attack - Brute Force', 131),
 ('BENIGN', 2000),
 ('DDoS-LOIC-UDP', 2000),
 ('Botnet Ares', 2000),
 ('Infiltration - Communication Victim Attacker', 204),
 ('DoS Slowloris', 2000),
 ('DoS GoldenEye', 2000),
 ('SSH-BruteForce', 2000),
 ('DoS Hulk', 2000),
 ('DDoS-LOIC-HTTP', 2000),
 ('Infiltration - Dropbox Download', 85),
 ('Web Attack - SQL', 39)]

In [9]:
con.execute('SELECT DISTINCT Timestamp, count(Timestamp) FROM flows GROUP BY Timestamp ORDER BY count(Timestamp) DESC')
con.fetchall()

[(datetime.datetime(2018, 2, 23, 19, 9, 2, 376264), 4),
 (datetime.datetime(2018, 2, 23, 20, 26, 27, 227348), 4),
 (datetime.datetime(2018, 3, 2, 19, 58, 38, 810871), 4),
 (datetime.datetime(2018, 3, 2, 13, 2, 9, 861636), 4),
 (datetime.datetime(2018, 2, 16, 19, 39, 18, 987763), 4),
 (datetime.datetime(2018, 2, 28, 19, 13, 37, 662398), 4),
 (datetime.datetime(2018, 2, 28, 14, 59, 56, 608452), 4),
 (datetime.datetime(2018, 2, 20, 16, 48, 4, 377943), 4),
 (datetime.datetime(2018, 2, 21, 14, 6, 43, 371068), 3),
 (datetime.datetime(2018, 2, 22, 16, 29, 22, 450529), 3),
 (datetime.datetime(2018, 3, 1, 13, 10, 47, 776575), 3),
 (datetime.datetime(2018, 2, 20, 21, 51, 10, 878301), 3),
 (datetime.datetime(2018, 2, 23, 15, 3, 22, 435968), 3),
 (datetime.datetime(2018, 2, 16, 17, 48, 7, 833857), 3),
 (datetime.datetime(2018, 3, 1, 18, 45, 15, 535781), 3),
 (datetime.datetime(2018, 2, 23, 20, 26, 18, 278960), 3),
 (datetime.datetime(2018, 2, 28, 19, 13, 38, 688697), 3),
 (datetime.datetime(2018, 

In [10]:
con.execute('CREATE OR REPLACE TABLE CSECICIDS2018 ("Protocol" BIGINT, "Flow Duration" BIGINT, "Total Fwd Packet" BIGINT, "Total Bwd packets" BIGINT, "Total Length of Fwd Packet" BIGINT, "Total Length of Bwd Packet" BIGINT, "Fwd Packet Length Max" BIGINT, "Fwd Packet Length Min" BIGINT, "Fwd Packet Length Mean" DOUBLE, "Fwd Packet Length Std" DOUBLE, "Bwd Packet Length Max" BIGINT, "Bwd Packet Length Min" BIGINT, "Bwd Packet Length Mean" DOUBLE, "Bwd Packet Length Std" DOUBLE, "Flow Bytes/s" DOUBLE, "Flow Packets/s" DOUBLE, "Flow IAT Mean" DOUBLE, "Flow IAT Std" DOUBLE, "Flow IAT Max" BIGINT, "Flow IAT Min" BIGINT, "Fwd IAT Total" BIGINT, "Fwd IAT Mean" DOUBLE, "Fwd IAT Std" DOUBLE, "Fwd IAT Max" BIGINT, "Fwd IAT Min" BIGINT, "Bwd IAT Total" BIGINT, "Bwd IAT Mean" DOUBLE, "Bwd IAT Std" DOUBLE, "Bwd IAT Max" BIGINT, "Bwd IAT Min" BIGINT, "Fwd PSH Flags" BIGINT, "Bwd PSH Flags" BIGINT, "Fwd URG Flags" BIGINT, "Bwd URG Flags" BIGINT, "Fwd RST Flags" BIGINT, "Bwd RST Flags" BIGINT, "Fwd Header Length" BIGINT, "Bwd Header Length" BIGINT, "Fwd Packets/s" DOUBLE, "Bwd Packets/s" DOUBLE, "Packet Length Min" BIGINT, "Packet Length Max" BIGINT, "Packet Length Mean" DOUBLE, "Packet Length Std" DOUBLE, "Packet Length Variance" DOUBLE, "FIN Flag Count" BIGINT, "SYN Flag Count" BIGINT, "RST Flag Count" BIGINT, "PSH Flag Count" BIGINT, "ACK Flag Count" BIGINT, "URG Flag Count" BIGINT, "CWR Flag Count" BIGINT, "ECE Flag Count" BIGINT, "Down/Up Ratio" DOUBLE, "Average Packet Size" DOUBLE, "Fwd Segment Size Avg" DOUBLE, "Bwd Segment Size Avg" DOUBLE, "Fwd Bytes/Bulk Avg" BIGINT, "Fwd Packet/Bulk Avg" BIGINT, "Fwd Bulk Rate Avg" BIGINT, "Bwd Bytes/Bulk Avg" BIGINT, "Bwd Packet/Bulk Avg" BIGINT, "Bwd Bulk Rate Avg" BIGINT, "Subflow Fwd Packets" BIGINT, "Subflow Fwd Bytes" BIGINT, "Subflow Bwd Packets" BIGINT, "Subflow Bwd Bytes" BIGINT, "FWD Init Win Bytes" BIGINT, "Bwd Init Win Bytes" BIGINT, "Fwd Act Data Pkts" BIGINT, "Fwd Seg Size Min" BIGINT, "Active Mean" DOUBLE, "Active Std" DOUBLE, "Active Max" BIGINT, "Active Min" BIGINT, "Idle Mean" DOUBLE, "Idle Std" DOUBLE, "Idle Max" BIGINT, "Idle Min" BIGINT, "ICMP Code" BIGINT, "ICMP Type" BIGINT, "Total TCP Flow Time" BIGINT, "Label" VARCHAR(255), "Attempted Category" BIGINT )')

In [22]:
con.execute('WITH tab1 AS (SELECT Flow_ID FROM sampled_flows) INSERT INTO CSECICIDS2018 SELECT * EXCLUDE (id, Flow_ID, "Flow ID", "Src IP","Src Port","Dst IP","Dst Port",Timestamp) FROM read_csv_auto("../datasets/CSECICIDS2018_improved/*.csv") INNER JOIN tab1 ON Flow_ID = "Flow ID"')

In [24]:
con.execute("UPDATE CSECICIDS2018 SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [25]:
con.execute('SELECT Label, COUNT(*) AS count FROM CSECICIDS2018 GROUP BY Label')
con.fetchall()

[('Web Attack - XSS', 119),
 ('DDoS-HOIC', 15189),
 ('Web Attack - Brute Force', 133),
 ('Infiltration - NMAP Portscan', 4047),
 ('BENIGN', 13259),
 ('DDoS-LOIC-UDP', 28300),
 ('Botnet Ares', 2117),
 ('Infiltration - Communication Victim Attacker', 17610),
 ('DoS Slowloris', 12828),
 ('DoS GoldenEye', 3547),
 ('SSH-BruteForce', 13453),
 ('DoS Hulk', 260287),
 ('DDoS-LOIC-HTTP', 3716),
 ('Web Attack - SQL', 39),
 ('Infiltration - Dropbox Download', 419)]

In [58]:
con.execute("COPY (SELECT * FROM CSECICIDS2018_SAMPLED) TO '../datasets/CSECICIDS2018_improved.parquet' (FORMAT PARQUET)")